In [1]:
import sys
sys.path.append("E:/Project/ZLTProgram/CelebA")
from Utils.dataload import CELEBA,CELEBA_Attack,drawCelebAImages
from torchvision import transforms
import torch
from Model.function import sample_z
import numpy as np
# 数据集存储目录
root='E:/Project/ModelAndDataset/data'
attr_name='Smiling'
batch_size=64
# 模型参数存放目录
save_model_dir='E:/Project/ModelAndDataset/model/CelebA/cVAE_GAN'
rawDataset = CELEBA_Attack(root="E:/Project/ModelAndDataset/data/celebA/Attack",
                        adv=False,
                        attackMethod="FastGradientMethod",
                        model='ResNet18',
                        label='Smiling',
                        transform=transforms.ToTensor()
                        )
rawLoader = torch.utils.data.DataLoader(rawDataset, batch_size=batch_size, shuffle=False)

advDataset=CELEBA_Attack(root="E:/Project/ModelAndDataset/data/celebA/Attack",
                        adv=True,
                        attackMethod="FastGradientMethod",
                        model='ResNet18',
                        label='Smiling',
                        transform=transforms.ToTensor()
                        )
advLoader=torch.utils.data.DataLoader(advDataset,batch_size=batch_size,shuffle=False)

Load images from: E:/Project/ModelAndDataset/data/celebA/Attack/RawImage_FastGradientMethod_ResNet18_Smiling.npy
Load labels from: E:/Project/ModelAndDataset/data/celebA/Attack/RawLabel_FastGradientMethod_ResNet18_Smiling.npy
Load images from: E:/Project/ModelAndDataset/data/celebA/Attack/AdvImage_FastGradientMethod_ResNet18_Smiling.npy
Load labels from: E:/Project/ModelAndDataset/data/celebA/Attack/AdvLabel_FastGradientMethod_ResNet18_Smiling.npy


In [2]:
from Model.CVAE_GAN_AdvancedV2 import CVAE
from os.path import join
#GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
nz=100
fsize=64
cvae = CVAE(nz=nz,
			 imSize=64,
			 enc_self_attn=True,
			 dec_self_attn=True,
			 g_spectral_norm=False,
			 CBN=True,
			 fSize=fsize,
			 device=device)
cvae.to(device)
cvae.load_params(modelDir=join(save_model_dir,attr_name))
cvae.eval()

loading params...


CVAE(
  (enc1): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (enc2): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (enc3): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (enc4): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (enc_attn_block): Self_Attn(
    (conv1x1_theta): Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_phi): Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_g): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_attn): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (softmax): Softmax(dim=-1)
  )
  (encLogVar): Linear(in_features=8192, out_features=100, bias=True)
  (encMu): Linear(in_features=8192, out_features=100, bias=True)
  (encY): Linear(in_features=8192, out_features=2, bias=True)
  (

In [3]:
pics,y=next(iter(rawLoader))

mu, log_var, _= cvae.encode(pics)
z = cvae.re_param(mu, log_var)
# 生成独热编码矩阵
yRand = torch.eye(2)[torch.LongTensor(y.data.cpu().numpy())].type_as(z)
yOnes = torch.eye(2)[torch.LongTensor(np.ones(y.size()).astype(int))].type_as(z)
yZeros = torch.eye(2)[torch.LongTensor(np.zeros(y.size()).astype(int))].type_as(z)

gen_pics=cvae.decode(y=yRand.to(device),z=z.to(device))
gen_pics0=cvae.decode(y=yZeros.to(device),z=z.to(device))
gen_pics1=cvae.decode(y=yOnes.to(device),z=z.to(device))

recon_x,_,_,_=cvae(pics)

drawCelebAImages(pics.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/real_image.png",
                 overwrite=True
                 )

drawCelebAImages(gen_pics.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/imgae_label.png",
                 overwrite=True
                 )
drawCelebAImages(gen_pics0.detach().cpu().numpy(),
                 np.zeros(y.size()).astype(int),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/negative.png",
                 overwrite=True
                 )
drawCelebAImages(gen_pics1.detach().cpu().numpy(),
                 np.ones(y.size()).astype(int),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/positive.png",
                 overwrite=True
                 )
drawCelebAImages(recon_x.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/recon_img.png",
                 overwrite=True
                 )







e:\Project\ZLTProgram\CelebA\Model\CVAE_GAN_AdvancedV2.py:159: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.encY(x.detach()))


In [13]:
diff=recon_x.cpu()-gen_pics.cpu()

drawCelebAImages(((diff - diff.min()) / (diff.max() - diff.min())).detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/difference.png",
                 overwrite=True
                 )


In [9]:
adv_pics,adv_y=next(iter(advLoader))

mu, log_var, _= cvae.encode(adv_pics)
z = cvae.re_param(mu, log_var)
# 生成独热编码矩阵
yRand = torch.eye(2)[torch.LongTensor(adv_y.data.cpu().numpy())].type_as(z)
yOnes = torch.eye(2)[torch.LongTensor(np.ones(adv_y.size()).astype(int))].type_as(z)
yZeros = torch.eye(2)[torch.LongTensor(np.zeros(adv_y.size()).astype(int))].type_as(z)

adv_gen_pics=cvae.decode(y=yRand.to(device),z=z.to(device))
adv_gen_pics0=cvae.decode(y=yZeros.to(device),z=z.to(device))
adv_gen_pics1=cvae.decode(y=yOnes.to(device),z=z.to(device))

adv_recon_x,_,_,_=cvae(adv_pics)

drawCelebAImages(adv_pics.detach().cpu().numpy(),
                 adv_y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/real_image_adv.png",
                 overwrite=True
                 )

drawCelebAImages(adv_gen_pics.detach().cpu().numpy(),
                 adv_y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/imgae_label_adv.png",
                 overwrite=True
                 )
drawCelebAImages(adv_gen_pics0.detach().cpu().numpy(),
                 np.zeros(y.size()).astype(int),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/negative_adv.png",
                 overwrite=True
                 )
drawCelebAImages(adv_gen_pics1.detach().cpu().numpy(),
                 np.ones(y.size()).astype(int),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/positive_adv.png",
                 overwrite=True
                 )
drawCelebAImages(adv_recon_x.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/recon_img_adv.png",
                 overwrite=True
                 )



e:\Project\ZLTProgram\CelebA\Model\CVAE_GAN_AdvancedV2.py:159: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.encY(x.detach()))


In [12]:
diff=adv_recon_x.cpu()-adv_gen_pics.cpu()
drawCelebAImages(((diff - diff.min()) / (diff.max() - diff.min())).detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/difference_adv.png",
                 overwrite=True
                 )
